<a href="https://colab.research.google.com/gist/jbdamask/8d53d0b017fc8cb8b3d8a20be6bc8d92/triage_example_auto_routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%
!pip install --quiet "openai>=1.14.0" "pydantic>=2.6.0"


In [2]:
# %%
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Callable, Any, Dict   # ← Callable is imported here
import os, json, textwrap
import pandas as pd
from google.colab import userdata, files
from enum import Enum

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [3]:
class Dog(BaseModel):
    breed: str = Field(..., description="Common breed name, e.g. Labrador Retriever")
    average_size: str = Field(..., description="Typical adult size (height or weight)")
    personality: str = Field(..., description="Brief temperament description")


In [4]:
class Route(str, Enum):
    structured_dog = "structured_dog"
    search = "search"
    download = "download"

class TriageDecision(BaseModel):
    route: Route

In [5]:
RouteHandler = Callable[[str], Any]
ROUTE_TABLE: Dict[Route, RouteHandler] = {}

def register(route: Route) -> Callable[[RouteHandler], RouteHandler]:
    """Decorator that registers a handler for a route."""
    def _wrap(fn: RouteHandler) -> RouteHandler:
        ROUTE_TABLE[route] = fn
        return fn
    return _wrap

In [6]:
@register(Route.structured_dog)
# def get_dog_info(breed_query: str) -> Dog:
def _handle_structured_dog(user_query: str):
    """
    Ask the model for breed details and validate with Pydantic.
    """
    # schema = Dog.model_json_schema()
    resp = client.responses.parse(
        model="gpt-4o-mini",
        input=f"Describe the dog breed in this query: '{user_query}'.",
        text_format=Dog,
    )
    raw_json = resp.output[0].content[0].text
    return Dog.model_validate_json(raw_json)


In [7]:
@register(Route.search)
def _handle_search(user_query: str) -> str:
    """
    Perform a real-time web search and return the model’s answer.
    """
    resp = client.responses.create(
        model="gpt-4o-mini",
        input=user_query,
        tools=[{"type": "web_search"}],  # let the model invoke it

    )

    return resp.output[1].content[0].text  # text answer follows the web_search_call


In [8]:
@register(Route.download)
def _handle_download(_: str):
    """
    Save some data
    """
    data = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
    df = pd.DataFrame(data)
    df.to_csv('output.csv', index=False)
    print("downloading your file")
    files.download('output.csv')
    return ("Check your download folder for output.csv")

In [9]:
def triage_query(user_query: str) -> Route:
    """
    Ask the model what the user wants.
    """
    system = textwrap.dedent(
        """
        You are a router.
        Your only job is to determine which route best matches the user's query.
        Note that certain queries may imply multiple routes. In such cases,
        choose the primary route that best fits the main intent of the query.
        Respond ONLY with valid JSON.
        """
    )
    resp = client.responses.parse(
        model="gpt-4.1-nano",
        input=[{"role": "developer", "content": system},
               {"role": "user", "content": user_query}],
        text_format=TriageDecision,
    )
    print(TriageDecision.model_validate_json(resp.output[0].content[0].text).route)
    return TriageDecision.model_validate_json(resp.output[0].content[0].text).route

In [10]:
def smart_answer(user_query: str):
    route   = triage_query(user_query)
    handler = ROUTE_TABLE.get(route, ROUTE_TABLE[Route.search])  # default
    return handler(user_query)

In [11]:
# %%
if __name__ == "__main__":
    print("Structured run →", smart_answer("Tell me about the Bernese Mountain Dog"))
    print("Search run      →", smart_answer("What are the top VC deals this week?"))
    print("Save file ", smart_answer("Save it!"))


Route.structured_dog
Structured run → breed='Bernese Mountain Dog' average_size='70-115 lbs (31-52 kg)' personality='Gentle, affectionate, and good-natured; friendly with children and other pets.'
Route.search
Search run      → As of April 25, 2025, several significant venture capital (VC) deals have been reported:

- **Butterfly Effect** raised $75 million to advance its initiatives. ([axios.com](https://www.axios.com/newsletters/axios-pro-rata-54c11523-671b-4a53-9739-99b0e4688ad5?utm_source=openai))

- **Fora** secured $40 million in a Series C funding round. ([axios.com](https://www.axios.com/newsletters/axios-pro-rata-54c11523-671b-4a53-9739-99b0e4688ad5?utm_source=openai))

- **Reducto** completed a $24.5 million Series A funding round. ([axios.com](https://www.axios.com/newsletters/axios-pro-rata-54c11523-671b-4a53-9739-99b0e4688ad5?utm_source=openai))

- **Supabase** raised $200 million to support its growth. ([axios.com](https://www.axios.com/newsletters/axios-pro-rata-633287cb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save file  Check your download folder for output.csv
